In [ ]:
from minio import Minio
from minio.error import S3Error

In [ ]:
import pandas as pd
import unicodedata

from psycopg2 import sql
from psycopg2.extras import execute_batch
import psycopg2

In [ ]:
def conectar_minio(endpoint='localhost:9000', 
                  access_key=None, 
                  secret_key=None, 
                  secure=False):
    """
    Establece una conexión con un servidor MinIO.
    
    Parámetros:
    -----------
    endpoint : str, opcional
        Dirección del servidor MinIO (por defecto 'localhost:9000')
    access_key : str, requerido
        Usuario/access key para la autenticación
    secret_key : str, requerido
        Contraseña/secret key para la autenticación
    secure : bool, opcional
        Si es True, usa HTTPS. Si es False, usa HTTP (por defecto False)
    
    Retorna:
    --------
    Minio
        Cliente MinIO conectado
    
    Excepciones:
    ------------
    ValueError
        Si no se proporcionan access_key o secret_key
    S3Error
        Si hay un error al conectar con el servidor MinIO
    """
    
    if not access_key or not secret_key:
        raise ValueError("Se requieren access_key y secret_key para la conexión")
    
    try:
        # Crear cliente MinIO
        cliente = Minio(
            endpoint,
            access_key=access_key,
            secret_key=secret_key,
            secure=secure
        )
        
        # Verificar conexión listando los buckets (opcional)
        buckets = cliente.list_buckets()
        print(f"Conexión exitosa a MinIO en {endpoint}")
        print(f"Buckets disponibles: {[bucket.name for bucket in buckets]}")
        
        return cliente
    
    except S3Error as err:
        print(f"Error al conectar con MinIO: {err}")
        raise

Leer archivo desde minio Minio

In [ ]:
import PyPDF2
import docx
from io import BytesIO

def extraer_archivo_minio(minio_client, bucket_name, ruta_archivo, tipo_archivo):
    """
    Extrae un archivo de MinIO y lo retorna en el formato adecuado.
    
    Parámetros:
    -----------
    minio_client : Minio
        Cliente MinIO ya conectado
    bucket_name : str
        Nombre del bucket donde está el archivo
    ruta_archivo : str
        Ruta completa del archivo dentro del bucket
    tipo_archivo : str
        Tipo de archivo a extraer ('pdf', 'word', 'excel', 'csv', 'parquet')
    
    Retorna:
    --------
    Depende del tipo de archivo:
    - 'pdf': Texto extraído (str)
    - 'word': Documento de python-docx
    - 'excel': DataFrame de pandas
    - 'csv': DataFrame de pandas
    - 'parquet': DataFrame de pandas
    
    Excepciones:
    ------------
    ValueError
        Si los parámetros son inválidos o el tipo no es soportado
    S3Error
        Si hay un error al acceder al archivo o el bucket no existe
    """
    
    # Validaciones iniciales
    if not minio_client:
        raise ValueError("Se requiere un cliente MinIO válido")
    if not bucket_name or not ruta_archivo or not tipo_archivo:
        raise ValueError("bucket_name, ruta_archivo y tipo_archivo son requeridos")
    
    tipo_archivo = tipo_archivo.lower()
    tipos_soportados = ['pdf', 'word', 'excel', 'csv', 'parquet']
    
    if tipo_archivo not in tipos_soportados:
        raise ValueError(f"Tipo de archivo '{tipo_archivo}' no soportado. Use: {tipos_soportados}")
    
    try:
        # Obtener el objeto de MinIO
        response = minio_client.get_object(bucket_name, ruta_archivo)
        data = BytesIO(response.read())
        data.seek(0)
        
        # Procesar según el tipo de archivo
        if tipo_archivo == 'pdf':
            # Extraer texto de PDF
            pdf_reader = PyPDF2.PdfReader(data)
            text = "\n".join([page.extract_text() for page in pdf_reader.pages])
            return text
        
        elif tipo_archivo == 'word':
            # Retornar documento de Word
            return docx.Document(data)
        
        elif tipo_archivo in ['excel', 'csv', 'parquet']:
            # Leer con pandas según el formato
            if tipo_archivo == 'excel':
                return pd.read_excel(data)
            elif tipo_archivo == 'csv':
                return pd.read_csv(data)
            elif tipo_archivo == 'parquet':
                return pd.read_parquet(data)
        
    except S3Error as err:
        print(f"Error al acceder al archivo en MinIO: {err}")
        raise
    except Exception as e:
        print(f"Error al procesar el archivo {ruta_archivo}: {e}")
        raise
    finally:
        response.close()
        response.release_conn()

Generar ruta 

In [ ]:
from datetime import datetime

def generar_ruta_fecha(separador='/', fecha=None):
    """
    Genera un string con la fecha en formato año/mes/día.
    
    Parámetros:
    -----------
    separador : str, opcional
        Carácter separador entre componentes (por defecto '/')
    fecha : datetime, opcional
        Fecha específica a formatear (si None, usa fecha actual)
    
    Retorna:
    --------
    str
        String con el formato 'YYYY{separador}MM{separador}DD'
    """
    # Usar fecha actual si no se proporciona una específica
    fecha_a_usar = fecha if fecha is not None else datetime.now()
    
    # Formatear la fecha
    ruta_fecha = fecha_a_usar.strftime(f"%Y{separador}%m{separador}%d")
    
    return ruta_fecha

Interacciones con la base de datos

In [ ]:
def get_table_columns(table_name, connection_params):
    """
    Obtiene las columnas de una tabla en PostgreSQL.

    Args:
        table_name (str): Nombre de la tabla en PostgreSQL.
        connection_params (dict): Parámetros de conexión a PostgreSQL.

    Returns:
        list: Lista con los nombres de las columnas de la tabla.
    """
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()

        # Consultar los nombres de las columnas de la tabla
        query = f"""
            SELECT column_name
            FROM information_schema.columns
            WHERE table_name = '{table_name}';
        """
        cursor.execute(query)
        columns = [row[0] for row in cursor.fetchall()]

        return columns

    except Exception as e:
        print(f"Error al obtener las columnas de la tabla {table_name}: {e}")
        return []
    finally:
        if conn:
            cursor.close()
            conn.close()

In [ ]:
def insert_into_info_lab_difuntos(df, connection_params, table_name='infolab_difuntos'):
    """
    Inserta los datos de un DataFrame en la tabla 'infolab' de PostgreSQL,
    considerando solo las columnas que coincidan entre los nombres del DataFrame y los de la tabla.

    Args:
        df (pd.DataFrame): DataFrame con los datos a insertar.
        connection_params (dict): Diccionario con los parámetros de conexión a PostgreSQL.
    """
    # Obtener las columnas de la tabla 'infolab'
    table_columns = get_table_columns(table_name, connection_params)
    
    if not table_columns:
        raise ValueError("No se pudieron obtener las columnas de la tabla 'infolab'.")    
    
    #if not matching_columns:
    #    raise ValueError("No hay columnas coincidentes entre el DataFrame y la tabla 'infolab'.")
    
    # Conexión a PostgreSQL    
    try:
        conn = psycopg2.connect(**connection_params)
        cursor = conn.cursor()
        
        # Crea la columna id_paciente dentro del df
        mapeo_columnas = {
                'tipo_de_identificacion': 'tipo_de_identificacion',
                'numero_de_identificacion_personal': 'identificacion',
            }
        df = df.rename(columns=mapeo_columnas)
        df = limpiar_y_validar_id_paciente(df)        

        # Filtrar solo registros cuyo id_paciente exista en tabla paciente
        ids_candidatos = df['id_paciente'].dropna().unique().tolist()
        if not ids_candidatos:
            print("No hay id_paciente válidos para insertar en info_lab.")
            return {'insertados': 0}

        cursor.execute("SELECT id_paciente FROM paciente WHERE id_paciente = ANY(%s)", (ids_candidatos,))
        existentes = {row[0] for row in cursor.fetchall()}
        if not existentes:
            print("No se encontraron id_paciente existentes en la tabla paciente.")
            return {'insertados': 0}

        df = df[df['id_paciente'].isin(existentes)].copy()

        # Seleccionar las columnas que coincidan entre el DataFrame y la tabla
        matching_columns = [col for col in df.columns if col in table_columns]
        
        # Crear la consulta INSERT dinámica
        columns_str = ", ".join(matching_columns)
        placeholders_str = ", ".join(["%s"] * len(matching_columns))
        insert_query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders_str})"

        # Preparar los datos para la inserción
        #values = df[matching_columns].values.tolist()
        values = df[matching_columns].where(pd.notna(df[matching_columns]), None).values.tolist()

        # Ejecutar la consulta
        cursor.executemany(insert_query, values)
        conn.commit()

        print(f"Se insertaron {cursor.rowcount} filas en la tabla {table_name}.")
    
    except Exception as e:
        print(f"Error al insertar los datos: {e}")
        conn.rollback()
    finally:
        if conn:
            cursor.close()
            conn.close()

Tratamiento de datos

In [ ]:
def normalizar_01(texto):
    if pd.isna(texto):
        return ''
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ascii', 'ignore').decode('utf-8')
    return texto.lower()

In [ ]:
def concatenar_columnas(df, columnas, nueva_columna,separador=' | '):
    df[nueva_columna] = (
        df[columnas]
        .fillna('') # reemplaza NaN por vacío
        .apply(lambda row: separador.join(str(v).strip() for v in row if str(v).strip() != ''), axis=1)
    )

In [ ]:
# Función mascara para valores inválidos dentro de dataframes
def es_valor_invalido(x):
    if pd.isna(x) or x is None:
        return True
    x_str = str(x).strip().upper()
    return x_str in ['NAN', 'NONE', 'NULL', '', 'SIN INFORMACIÓN', 'N/A']

In [ ]:
def limpiar_y_validar_id_paciente(df_para_insertar):

    # Validacioneas para crear id_paciente
    mask_tipo_id_valido = ~df_para_insertar['tipo_de_identificacion'].apply(es_valor_invalido)            
    mask_id_valido = ~df_para_insertar['identificacion'].apply(es_valor_invalido)
    df_para_insertar = df_para_insertar[mask_tipo_id_valido & mask_id_valido].copy()
    df_para_insertar['tipo_de_identificacion'] = df_para_insertar['tipo_de_identificacion'].astype(str).str.strip().str.upper()
    df_para_insertar['identificacion'] = df_para_insertar['identificacion'].astype(str).str.strip()
    df_para_insertar['id_paciente'] = df_para_insertar['tipo_de_identificacion'] + df_para_insertar['identificacion']
    df_para_insertar = df_para_insertar.dropna(subset=['id_paciente'])
    print(df_para_insertar.columns)

    return df_para_insertar